In [3]:
#Phase 7: GNN Model for Protein-Ligand Binding Affinity Prediction

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Dataset
import pandas as pd
import numpy as np
import pickle
from pathlib import Path


class model(nn.Module):
    def __init__(self, nodeFeatureDimension, proteinEmbeddingDimension, hiddenDimension = 128, gnnLayers = 3, outputDimension = 1):
        super(model, self).__init__()

        #Layers
        self.conv1 = GCNConv(nodeFeatureDimension, hiddenDimension)
        self.conv2 = GCNConv(hiddenDimension, hiddenDimension)
        self.conv3 = GCNConv(hiddenDimension, hiddenDimension)

        #Multilayer Perceptron for prediction (combining gnn output with protein embeddings)
        combinedDimension = hiddenDimension + proteinEmbeddingDimension
        self.fc1 = nn.Linear(combinedDimension, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, outputDimension)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, graphData, proteinEmbedding):
        x = graphData.x
        edgeIndex = graphData.edge_index
        batch = graphData.batch

        #gnn layers w/ ReLu activation
        x = self.conv1(x, edgeIndex)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv2(x, edgeIndex)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv3(x, edgeIndex)
        x = F.relu(x)

        #global pooling
        x = global_mean_pool(x, batch)

        #concatenating gnn embedding w/ protein embedding
        combined = torch.cat([x, proteinEmbedding], dim=1)

        #multilayer perceptron for actual prediction
        x = self.fc1(combined)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        return x
    
class proteinLigandDataset(Dataset):
    def __init__(self, df, ligandGraphs, proteinEmbeddings):
        super(proteinLigandDataset, self).__init__()
        self.df = df
        self.ligandGraphs = ligandGraphs
        self.proteinEmbeddings = proteinEmbeddings

    def len(self):
        return len(self.df)
        
    def get(self, index):
        row = self.df.iloc[index]

        #SMILES and protein sequences
        SMILES = row['Ligand SMILES']
        proteinSequence = row['BindingDB Target Chain Sequence 1']
        label = row['pKd']

        #pull cached embeddings
        graphDictionary = self.ligandGraphs[SMILES]
        proteinEmb = self.proteinEmbeddings[proteinSequence]

        #convert graph dictionaries to pytorch geometric data objects
        nodeFeatures = torch.tensor(graphDictionary['nodeFeatures'], dtype=torch.float)
        edgeIndex = torch.tensor(graphDictionary['edgeIndex'], dtype=torch.long).t().contiguous()

        #create data object
        graphData = Data(x = nodeFeatures, edge_index = edgeIndex)

        #convert protein embeddings to tensors
        proteinTensor = torch.tensor(proteinEmb, dtype=torch.float).unsqueeze(0)
        labelTensor = torch.tensor([label], dtype=torch.float)

        return graphData, proteinTensor, labelTensor

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_scatter/_version_cpu.so, 0x0006): symbol not found in flat namespace '__ZN3c1017RegisterOperatorsD1Ev'
  import torch_geometric.typing
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_cluster/_version_cpu.so, 0x0006): symbol not found in flat namespace '__ZN3c1017RegisterOperatorsD1Ev'
  import torch_geometric.typing
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An i

In [ ]:
#loading embeddings
train_df = pd.read_csv('../data/splits/train.csv', low_memory=False)
val_df = pd.read_csv('../data/splits/val.csv', low_memory=False)
test_df = pd.read_csv('../data/splits/test.csv', low_memory=False)

# Load cached ligand graphs
with open('../embeddings/ligands/ligand_graphs.pkl', 'rb') as f:
    ligandGraphs = pickle.load(f)

# Load cached protein embeddings
with open('../embeddings/proteins/protein_embeddings.pkl', 'rb') as f:
    proteinEmbeddings = pickle.load(f)

In [ ]:
# create datasets and dataloaders
from torch_geometric.loader import DataLoader

train_dataset = proteinLigandDataset(train_df, ligandGraphs, proteinEmbeddings)
val_dataset = proteinLigandDataset(val_df, ligandGraphs, proteinEmbeddings)
test_dataset = proteinLigandDataset(test_df, ligandGraphs, proteinEmbeddings)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# get feature dimensions from sample
sample_graph, sample_protein, sample_label = train_dataset[0]
nodeFeatureDim = sample_graph.x.shape[1]
proteinEmbeddingDim = sample_protein.shape[1]

# initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gnn_model = model(nodeFeatureDim, proteinEmbeddingDim, hiddenDimension=128)
gnn_model = gnn_model.to(device)

In [ ]:
# Training function
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch_graph, batch_protein, batch_label in loader:
        batch_graph = batch_graph.to(device)
        batch_protein = batch_protein.to(device).squeeze(1)
        batch_label = batch_label.to(device)
        
        optimizer.zero_grad()
        predictions = model(batch_graph, batch_protein)
        loss = criterion(predictions, batch_label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_graph, batch_protein, batch_label in loader:
            batch_graph = batch_graph.to(device)
            batch_protein = batch_protein.to(device).squeeze(1)
            batch_label = batch_label.to(device)
            
            predictions = model(batch_graph, batch_protein)
            loss = criterion(predictions, batch_label)
            
            total_loss += loss.item()
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch_label.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))
    r2 = r2_score(all_labels, all_preds)
    
    return avg_loss, rmse, r2

# Import metrics
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Setup training
optimizer = torch.optim.Adam(gnn_model.parameters(), lr=0.001)
criterion = nn.MSELoss()
num_epochs = 50

print(f'Training for {num_epochs} epochs')
print('-' * 60)

best_val_rmse = float('inf')
train_losses = []
val_losses = []
val_rmses = []

# Training loop
for epoch in range(num_epochs):
    train_loss = train_epoch(gnn_model, train_loader, optimizer, criterion, device)
    val_loss, val_rmse, val_r2 = evaluate(gnn_model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_rmses.append(val_rmse)
    
    # Save best model
    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        torch.save(gnn_model.state_dict(), '../models/best_gnn_model.pt')
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val R²: {val_r2:.4f}')
